In [4]:
import pandas as pd
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [5]:
data = pd.read_csv('train_data.csv')
#break train data into train and validation
train_data, test_data = train_test_split(data, test_size=0.25, random_state=123) 

In [6]:
# Remove unnecessary columns
train_data = train_data.drop(['BookingID',], axis=1)
test_data = test_data.drop(['BookingID',], axis=1)

In [7]:
# Convert categorical features to numerical values using one-hot encoding
train_data = pd.get_dummies(train_data, columns=['MealPlan','RoomType', 'MarketSegment'])
test_data = pd.get_dummies(test_data, columns=['MealPlan','RoomType', 'MarketSegment'])

#meal plan 3 is not present in test data so we drop it from train data
train_data = train_data.drop(['MealPlan_Meal Plan 3'], axis=1)
test_data = test_data.drop(['MealPlan_Meal Plan 3'], axis=1)

In [8]:
print(train_data.columns)

Index(['LeadTime', 'ArrivalYear', 'ArrivalMonth', 'ArrivalDate',
       'NumWeekendNights', 'NumWeekNights', 'Parking', 'NumAdults',
       'NumChildren', 'RepeatedGuest', 'NumPrevCancellations',
       'NumPreviousNonCancelled', 'AvgRoomPrice', 'SpecialRequests',
       'BookingStatus', 'MealPlan_Meal Plan 1', 'MealPlan_Meal Plan 2',
       'MealPlan_Not Selected', 'RoomType_Room_Type 1', 'RoomType_Room_Type 2',
       'RoomType_Room_Type 3', 'RoomType_Room_Type 4', 'RoomType_Room_Type 5',
       'RoomType_Room_Type 6', 'RoomType_Room_Type 7',
       'MarketSegment_Aviation', 'MarketSegment_Complementary',
       'MarketSegment_Corporate', 'MarketSegment_Offline',
       'MarketSegment_Online'],
      dtype='object')


In [9]:
from sklearn.preprocessing import LabelEncoder

# Remove rows with missing values
train_data = train_data.dropna()
test_data = test_data.dropna()

# Split data into features and target variable
y_train = train_data['BookingStatus']
y_test = test_data['BookingStatus']

X_train = train_data.drop(['BookingStatus'], axis=1) 
X_test = test_data.drop(['BookingStatus'], axis=1)

# Convert string labels to numeric labels
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)



In [10]:
# Create the XGBoost classifier
xgb_model = xgb.XGBClassifier()

# Train the model on the training data
xgb_model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = xgb_model.predict(X_test)

# Evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.8934527911784976


In [11]:
import pandas as pd
check_data = pd.read_csv('check_data.csv')

empty_booking_status = check_data[check_data['BookingStatus'].isnull()]

# Remove unnecessary columns, including BookingStatus
input_data = empty_booking_status.drop(['BookingID'], axis=1)

# Convert categorical features to numerical values using one-hot encoding
input_data = pd.get_dummies(input_data, columns=['MealPlan', 'RoomType', 'MarketSegment'])

# Get the feature names used in the training data
train_features = train_data

# Check that the feature names in the input data match the feature names in the training data
if set(input_data.columns) != set(train_features):
    raise ValueError("Feature names in input data do not match feature names in training data")

In [12]:
# Check if the input data and the training data have the same columns
print("Input data columns:", set(input_data.columns))
print("Training data columns:", set(train_data.columns))

Input data columns: {'MealPlan_Meal Plan 2', 'RoomType_Room_Type 5', 'MealPlan_Meal Plan 1', 'MarketSegment_Offline', 'Parking', 'RoomType_Room_Type 2', 'RepeatedGuest', 'RoomType_Room_Type 6', 'NumPreviousNonCancelled', 'MarketSegment_Aviation', 'NumWeekendNights', 'RoomType_Room_Type 4', 'MarketSegment_Online', 'RoomType_Room_Type 1', 'NumChildren', 'AvgRoomPrice', 'NumAdults', 'LeadTime', 'NumWeekNights', 'MarketSegment_Complementary', 'MarketSegment_Corporate', 'ArrivalDate', 'NumPrevCancellations', 'RoomType_Room_Type 3', 'RoomType_Room_Type 7', 'MealPlan_Not Selected', 'ArrivalMonth', 'ArrivalYear', 'SpecialRequests', 'BookingStatus'}
Training data columns: {'MealPlan_Meal Plan 2', 'RoomType_Room_Type 5', 'MealPlan_Meal Plan 1', 'MarketSegment_Offline', 'Parking', 'RoomType_Room_Type 2', 'RepeatedGuest', 'RoomType_Room_Type 6', 'NumPreviousNonCancelled', 'MarketSegment_Aviation', 'NumWeekendNights', 'RoomType_Room_Type 4', 'MarketSegment_Online', 'RoomType_Room_Type 1', 'NumChild

In [18]:

# Make predictions on the input data
input_data = input_data.drop(['BookingStatus'], axis=1)
predictions = xgb_model.predict(input_data)

# Convert the predictions to a dataframe
predictions_df = pd.DataFrame(predictions)

ValueError: Feature shape mismatch, expected: 29, got 30